In [1]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

In [2]:
path_base = r"C:\Users\DellVostro\OneDrive\Documentos\XP\Projeto_aplicado\Enso\Dados_clima_RS"

In [3]:
def coleta_dados(path_base):
    dfs = []

    for root, dirs, files in os.walk(path_base):
        arquivos = [f for f in files if os.path.isfile(os.path.join(root, f))]
        primeiros_5 = sorted(arquivos)[:5]  # Ordena e seleciona os cinco primeiros

        for nome_arquivo in primeiros_5:
            caminho_base = os.path.join(root, nome_arquivo)
            try:
                df = pd.read_csv(
                    caminho_base,
                    sep=";",
                    header=8,
                    encoding="iso-8859-1",
                    usecols=[0, 2]
                )
                dfs.append(df)
            except Exception as e:
                print(f"Erro ao ler {caminho_base}: {e}")

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        print("Nenhum arquivo válido foi encontrado.")
        return pd.DataFrame()



In [4]:
df = coleta_dados(path_base)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759840 entries, 0 to 759839
Data columns (total 2 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   DATA (YYYY-MM-DD)                 759840 non-null  object
 1   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)  759840 non-null  object
dtypes: object(2)
memory usage: 11.6+ MB


### De 2019 em frente houve uma mudança no formato da data nas bases do INPE, apliquei a função só para essas datas e fiz a concatenação das bases após aplicar as modificações

In [6]:
path_base = r"C:\Users\DellVostro\OneDrive\Documentos\XP\Projeto_aplicado\Enso\Dados_clima_RS_2019_2024"

In [7]:
df_2019 = coleta_dados(path_base)

In [8]:
df_2019['Data'] = df_2019['Data'].str.replace("/", "-")

In [9]:
df_2019.rename({"Data": "DATA (YYYY-MM-DD)"}, axis = 1, inplace = True)

In [10]:
df = pd.concat([df, df_2019])

In [11]:
del df_2019

In [12]:
df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"] = df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].replace({"-9999": "0"})

In [13]:
df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"] = df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].str.replace(",", ".").astype(float)

In [26]:
import numpy as np
df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"] = np.where(df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"] == 0, np.nan, df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"])

In [14]:
df['Ano'] = df['DATA (YYYY-MM-DD)'].str[:4]

In [15]:
df.head()

,DATA (YYYY-MM-DD),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",Ano
0,2000-09-22,0.0,2000
1,2000-09-22,0.0,2000
2,2000-09-22,0.0,2000
3,2000-09-22,0.0,2000
4,2000-09-22,0.0,2000


In [16]:
from datetime import datetime
df["DATA (YYYY-MM-DD)"] =  pd.to_datetime(df["DATA (YYYY-MM-DD)"])

### Filtrando apenas os meses de outrubro a maio, em que ocorre o ciclo da soja no RS

In [17]:
df_meses = df[df["DATA (YYYY-MM-DD)"].dt.month.isin([10, 11, 12, 1, 2, 3, 4, 5])]

In [18]:
df_meses["DATA (YYYY-MM-DD)"].dt.month.unique()

array([10, 11, 12,  1,  2,  3,  4,  5])

### agrupando o total de precipitação por ano

In [33]:
df_media = df_meses[df_meses["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].notna()].groupby("Ano").sum(numeric_only= True)

In [38]:
df_media["precipitacao_media"] = df_media["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"]/5

In [43]:
df_media.reset_index(inplace = True)

### Dados de 2000 e 2001 estão muito fora do padrão e será excluídos

In [46]:
df_media = df_media[(df_media["Ano"] != "2000") & (df_media["Ano"] != "2001")]

In [48]:
df_media.to_csv("media_pluviometrica_RS.csv", sep = ";", index = False)